# New World Game Analyst Powered By Amazon Bedrock AgentCore

This notebook combines the functionality of the New World Game Analysis System, which includes specialized AI agents for analyzing game design documents and providing expert feedback about Amazon Game Studios' MMORPG "New World".

# System Overview

1. **Game Analyst Agent**: Main orchestrating agent that coordinates between specialized agents
2. **Lore Agent**: Specializes in game world, history, and narrative elements
3. **Gameplay Agent**: Focuses on game mechanics, systems, and player engagement
4. **Corporate Strategy Agent**: Focuses on company strategy and roadmaps
5. **QA Agent**: Focuses on player behavioral data and other in game data

# Data Sources

- Go over knowledge bases and breif on how they were set up and why you need them

# Prerequisites
- Python 3.10+
- AWS Credentials
- Amazon Bedrock AgentCore SDK
- Strands Agents

In [ ]:
# Install required packages
!uv add -r requirements.txt --active

## Load Credentials 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_PROFILE = os.getenv("AWS_PROFILE")
print(AWS_PROFILE)

# Lets build some simple agents

## Create Agent Roles

Create basic agent IAM roles with all permissions needed for the agents to be configured, launched, use tools such as knowledge bases, and invoke other agents

In [ ]:
from utils import create_agentcore_role
import os
import string
import random

def generate_random_suffix(length=6):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

gameplay_agent_name = f"gameplay_agent_{generate_random_suffix()}"
gameplay_iam_role = create_agentcore_role(agent_name=gameplay_agent_name, region=os.getenv("AWS_REGION"))
gameplay_role_arn = gameplay_iam_role['Role']['Arn']
gameplay_role_name = gameplay_iam_role['Role']['RoleName']
print(gameplay_role_arn)
print(gameplay_role_name)

lore_agent_name = f"lore_agent_{generate_random_suffix()}"
lore_agent_iam_role = create_agentcore_role(agent_name=lore_agent_name, region=os.getenv("AWS_REGION"))
lore_agent_role_arn = lore_agent_iam_role['Role']['Arn']
lore_agent_role_name = lore_agent_iam_role['Role']['RoleName']
print(lore_agent_role_arn)
print(lore_agent_role_name)

game_analyst_agent_name = f"game_analyst_agent_{generate_random_suffix()}"
game_analyst_iam_role = create_agentcore_role(agent_name=game_analyst_agent_name, region=os.getenv("AWS_REGION"))
game_analyst_role_arn = game_analyst_iam_role['Role']['Arn']
game_analyst_role_name = game_analyst_iam_role['Role']['RoleName']
print(game_analyst_iam_role)
print(game_analyst_role_arn)
print(game_analyst_role_name)

strategy_agent_name = f"strategy_agent_{generate_random_suffix()}"
strategy_iam_role = create_agentcore_role(agent_name=strategy_agent_name, region=os.getenv("AWS_REGION"))
strategy_role_arn = strategy_iam_role['Role']['Arn']
strategy_role_name = strategy_iam_role['Role']['RoleName']
print(strategy_iam_role)
print(strategy_role_arn)
print(strategy_role_name)


## Runtime Configuration

Helper function for configuring and launching AgentCore agent runtimes

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time


def configure_runtime(agent_name, agentcore_iam_role, python_file_name):
    boto_session = Session(region_name=os.getenv("AWS_REGION"))
    region = boto_session.region_name

    agentcore_runtime = Runtime()

    response = agentcore_runtime.configure(
        entrypoint=python_file_name,
        execution_role=agentcore_iam_role['Role']['Arn'],
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name=agent_name
    )
    return response, agentcore_runtime

def check_status(agent_runtime):
    status_response = agent_runtime.status()
    status = status_response.endpoint['status']
    end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
    while status not in end_status:
        time.sleep(10)
        status_response = agent_runtime.status()
        status = status_response.endpoint['status']
        print(status)
    return status

In [ ]:
# set the current working directory to be the gameplay-agent folder
import os
#os.chdir('../gameplay-agent')
os.chdir('./../gameplay-agent-basic')
print(os.getcwd())

## Create Gameplay Agent

In [ ]:
%%writefile gameplay_agent_no_memories.py

import argparse
import json
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Model: Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)


# Prompt: System prompt for the Gameplay agent
GAMEPLAY_AGENT_PROMPT = """
You are a New World Gameplay Expert specializing in all gameplay mechanics for New World built by Amazon Game Studios. Your expertise includes:
- Character attributes and weapon scaling (Strength, Dexterity, Intelligence, Focus)
- Crafting systems (Armoring, Weaponsmithing, Arcana, Cooking, etc.)
- Gathering and refining (Mining, Logging, Harvesting, Fishing, Skinning)
- Territory control and faction systems (Covenant, Marauders, Syndicate)
- Expeditions and PvE content
- Game mechanics and optimization strategies

ALWAYS use the available knowledge base tools to retrieve information before answering questions. If the knowledge base returns relevant information, use that information in your response.

When using knowledge base tools, make sure to:
1. Use the exact query or related terms to search the knowledge base
2. Include the retrieved information in your response
3. Only use information from the knowledge base
4. Provide practical gameplay advice based on the retrieved data
"""


app = BedrockAgentCoreApp()


# Create and run the Gameplay agent
@app.entrypoint
def gameplay_agent(payload):
    """
    Creates and runs a gameplay agent connected to the New World Knowledge Base
    """

    try:
        # Create a fresh MCP client for each invocation
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "new-world-design-docs"
                    }
                )
            )
        )
        
        with kb_client:
            # Get tools from the KB client
            kb_tools = kb_client.list_tools_sync()

            # Agent: Create the gameplay agent
            agent = Agent(
                tools=kb_tools,
                model=bedrock_model,
                system_prompt=GAMEPLAY_AGENT_PROMPT
            )

            user_input = payload.get("prompt")
            response = agent(user_input)
            output = {
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }
            return output

    except Exception as e:
        print(f"Error accessing Knowledge Base: {e}")
        return f"error {e}"

if __name__ == "__main__":
    app.run()


### Launch the gameplay agent runtime

In [ ]:
_, gameplay_agentcore_runtime = configure_runtime("gameplay_agent_no_memory", gameplay_iam_role, "gameplay_agent.py")
gameplay_launch_result = gameplay_agentcore_runtime.launch(auto_update_on_conflict="true")
gameplay_agent_id = gameplay_launch_result.agent_id
gameplay_agent_arn = gameplay_launch_result.agent_arn

print(gameplay_agent_arn)

### Save the gameplay agent ARN to AWS SSM ParameterStore

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/gameplay_agent_arn_no_memories',
    Value=gameplay_agent_arn,
    Type='String',
    Overwrite=True
)

### Check the gameplay agent's endpoint status

In [ ]:
status = check_status(gameplay_agentcore_runtime)
print(status)

### Test the gameplay agent

In [ ]:
invoke_response = gameplay_agentcore_runtime.invoke({"prompt": "What are factions in new world?"})
invoke_response

## Create the lore agent

The Lore Agent specializes in New World's game world, history, and narrative elements.

In [ ]:
# set the current working directory to be the tech_agent folder
os.chdir('../lore-agent-basic')
print(os.getcwd())

In [ ]:
%%writefile lore_agent_no_memories.py

from mcp import StdioServerParameters, stdio_client
import json
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

# System prompt for the Knowledge Base agent
LORE_AGENT_PROMPT = """
You are a Game Analyst expert specializing in game lore for the game New World built by Amazon Game Studios. Your expertise includes:
- Overview of what the game New World is and they main goals and story
- Lore and history of the world the game takes place in
- The game world and aspects such as locations, geography, demographics, and characters

You have access to a knowledge base containing information from the New World Fandom Wiki. ALWAYS use the available knowledge base tools to retrieve information before answering questions. If the knowledge base returns relevant information, use that information in your response.

When using knowledge base tools, make sure to:
1. Use the exact query or related terms to search the knowledge base
2. Include the retrieved information in your response
3. Only use information from the knowledge base
"""
app = BedrockAgentCoreApp()
# Create and run the Knowledge Base agent
@app.entrypoint
def lore_agent(payload):
    """
    Creates and runs an agent connected to the Knowledge Base
    """

    try:
        # Create a fresh MCP client for each invocation
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "new-world-design-docs"
                    }
                )
            )
        )
        
        with kb_client:
            # Get tools from the KB client
            kb_tools = kb_client.list_tools_sync()

            # Create the gameplay agent
            agent = Agent(
                tools=kb_tools,
                model=bedrock_model,
                system_prompt=LORE_AGENT_PROMPT
            )

            user_input = payload.get("prompt")
            response = agent(user_input)
            output = {
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }
            return output


    except Exception as e:
        print(f"Error accessing Knowledge Base: {e}")
        return f"error {e}"

if __name__ == "__main__":
    app.run()


### Launch the lore agent runtime

In [ ]:
_, lore_agentcore_runtime = configure_runtime("lore_agent_no_memories", lore_agent_iam_role, "lore_agent.py")
lore_launch_result = lore_agentcore_runtime.launch(auto_update_on_conflict="true")
lore_agent_id = lore_launch_result.agent_id
lore_agent_arn = lore_launch_result.agent_arn

print(lore_agent_arn)

### Save the lore agent ARN to AWS SSM ParameterStore

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/lore_agent_arn_no_memories',
    Value=lore_agent_arn,
    Type='String',
    Overwrite=True
)

### Save the Lore Agent ARN to Parameter Store

### Check the lore agent's endpoint status

In [ ]:
status = check_status(lore_agentcore_runtime)
print(status)

### Test the lore agent

In [ ]:
invoke_response = lore_agentcore_runtime.invoke({"prompt": "What is the origin of the Corruption in Aeternum and how is it connected to the ancient civilization that once inhabited the island?"})
invoke_response

## Create strategy agent

In [ ]:
# set the current working directory to be the game_analyst_agent folder
os.chdir('../strategy-agent-basic')
print(os.getcwd())

In [ ]:
%%writefile strategy_agent_no_memories.py

from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
import json
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

# System prompt for the Knowledge Base agent
STRATEGY_AGENT_PROMPT = """
You are a business strategy expert at Amazon Game Studios. Your expertise includes:
- Game industry market analysis and competitive positioning
- Player acquisition, retention, and monetization strategies
- Product portfolio management and roadmap planning
- Performance metrics analysis and strategic recommendations
- Resource allocation and investment prioritization
- Company priorities and roadmaps


You have access to a knowledge base containing corporate strategy documents of Amazon Game Studios. ALWAYS use the available knowledge base tools to retrieve information before answering questions. If the knowledge base returns relevant information, use that information in your response.

When using knowledge base tools, make sure to:
1. Use the exact query or related terms to search the knowledge base
2. Include the retrieved information in your response
3. Only use information from the knowledge base
"""
app = BedrockAgentCoreApp()
# Create and run the Knowledge Base agent
@app.entrypoint
def strategy_agent(payload):
    """
    Creates and runs an agent connected to the Knowledge Base
    """

    try:
        # Create a fresh MCP client for each invocation
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "strategy-docs"
                    }
                )
            )
        )
        
        with kb_client:
            # Get tools from the KB client
            kb_tools = kb_client.list_tools_sync()

            # Create the gameplay agent
            agent = Agent(
                tools=kb_tools,
                model=bedrock_model,
                system_prompt=STRATEGY_AGENT_PROMPT
            )

            user_input = payload.get("prompt")
            response = agent(user_input)
            output = {
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }
            return output


    except Exception as e:
        print(f"Error accessing Knowledge Base: {e}")
        return f"error {e}"

if __name__ == "__main__":
    app.run()



### Launch the strategy agent runtime

In [ ]:
_, strategy_agentcore_runtime = configure_runtime("strategy_agent_no_memories", strategy_iam_role, "strategy_agent.py")
strategy_launch_result = strategy_agentcore_runtime.launch(auto_update_on_conflict="true")
strategy_agent_id = strategy_launch_result.agent_id
strategy_agent_arn = strategy_launch_result.agent_arn

print(strategy_agent_arn)

### Save the strategy agent ARN to AWS SSM ParameterStore

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/strategy_agent_arn_no_memories',
    Value=strategy_agent_arn,
    Type='String',
    Overwrite=True
)

### Check the strategy agent's endpoint status

In [ ]:
status = check_status(strategy_agentcore_runtime)
print(status)

### Test the strategy agent

In [ ]:
invoke_response = strategy_agentcore_runtime.invoke({"prompt": "What is the completion rate of Tides of Corruption"})
invoke_response

## Create the Game Analyst Agent

The main orchestrating agent that coordinates between the specialized lore and gameplay agents.

In [ ]:
# set the current working directory to be the game_analyst_agent folder
print(os.getcwd())
os.chdir('./../game-analyst-agent-basic')
print(os.getcwd())

### Update the game analyst agent execution role to allow for agent runtime access

In [ ]:
# Let's update the game_analyst agentcore exeuction role so it has permissions to invoke the required subagents
# the game_analyst also needs needs permissions to retrieve the sub agent arns from parameter store
import json 

# retrieve the runtime arn from parameter store
ssm = boto3.client('ssm')
response = ssm.get_parameter(Name='/agents/gameplay_agent_arn_no_memories')
gameplay_agent_arn = response['Parameter']['Value']
gameplay_agent_parameter_arn = response['Parameter']['ARN']

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name='/agents/lore_agent_arn_no_memories')
lore_agent_arn = response['Parameter']['Value']
lore_agent_parameter_arn = response['Parameter']['ARN']

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name='/agents/strategy_agent_arn_no_memories')
strategy_agent_arn = response['Parameter']['Value']
strategy_agent_parameter_arn = response['Parameter']['ARN']

def update_game_analyst_permissions(sub_agent_arns: list, sub_agent_parameter_arns: list, game_analyst_name: str):
    iam_client = boto3.client('iam')
    game_analyst_permissions = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:InvokeAgentRuntime"
                ],
                "Resource": [ sub_agent_arn + "/runtime-endpoint/DEFAULT" for sub_agent_arn in sub_agent_arns ] + [ sub_agent_arn for sub_agent_arn in sub_agent_arns ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "ssm:GetParameter"
                ],
                "Resource": [sub_agent_parameter_arn for sub_agent_parameter_arn in sub_agent_parameter_arns]

            }]
    }
        
    rsp = iam_client.put_role_policy(
        RoleName=game_analyst_name,
        PolicyName="subagent_permissions-new",
        PolicyDocument=json.dumps(game_analyst_permissions)
    )
    return rsp

rsp = update_game_analyst_permissions([gameplay_agent_arn, lore_agent_arn, strategy_agent_arn], [gameplay_agent_parameter_arn, lore_agent_parameter_arn, strategy_agent_parameter_arn], game_analyst_role_name)
print(rsp)

### Update game analyst agent code

In [ ]:
%%writefile game_analyst_agent_no_memories.py

from strands import Agent, tool
from strands.models import BedrockModel
import argparse
import json
import logging
import boto3
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from invoke_agent_utils import invoke_agent_with_boto3

logger = logging.getLogger(__name__).setLevel(logging.DEBUG)

app = BedrockAgentCoreApp()

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

toolmetrics = []

# System prompt for the Game Analyst agent
GAME_ANALYST_PROMPT = """
PLEASE PRINT AT THE TOP IF YOU DONT USE THE TOOLS and EXPAND ON WHY YOU DIDNT. IF YOU DO USE THE TOOLS ANNOTATE WHERE.
You are a professional Game Analyst Agent specializing in analyzing submitted game design documents and providing expert feedback to game developers.
Your role is to carefully review game design documentation, understand the developer's vision, and answer their questions with insightful analysis and constructive recommendations.
When evaluating designs and content proposals provide feedback around lore, gameplay, and corporate strategy. All designs must be consistent with lore, gameplay, and corporate strategy.

You have access to the following tools that you can call upon when needed:
1. For queries around what the game is and questions about the game world and lore use lore_agent tool:
    - Specializes in narrative consistency, world-building, character development
    - Analyzes thematic elements, story structure, and narrative engagement
    - Evaluates lore systems, fictional histories, and setting coherence
    - Identifies opportunities for deeper player immersion through world design

2. For queries around game mechanics and gameplay use gameplay_agent tool:
    - Specializes in game systems, mechanics, balance, and player engagement
    - Analyzes core gameplay loops, progression systems, and player motivation
    - Evaluates difficulty curves, skill ceilings, and accessibility considerations
    - Identifies potential balance issues and optimization opportunities

3. For queries around corporate strategy, performance metrics, roadmaps, and strategic priorities use strategy_agent tool:
    - Specializes in corporate strategy, performance metrics, and strategic priorities
    - Analyzes corporate strategy, performance metrics, and strategic priorities
    - Evaluates corporate strategy, performance metrics, and strategic priorities
    - Identifies opportunities for improvement in corporate strategy, performance metrics, and strategic priorities
"""

def get_agent_arn(agent_name: str) -> str:
    """
    Retrieve agent ARN from Parameter Store
    """
    try:
        ssm = boto3.client('ssm')
        response = ssm.get_parameter(
            Name=f'/agents/{agent_name}_arn_no_memories'
        )
        return response['Parameter']['Value']
    except Exception as err:
        logger.exception("Exception getting ARNS")
        raise err

@tool
def get_lore_agent(payload) -> str:
    """
    Call the lore agent tool to get information about the game lore.
    """
    lore_agent_arn = get_agent_arn("lore_agent")
    response = json.loads(invoke_agent_with_boto3(lore_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']

@tool
def get_gameplay_agent(payload) -> str:
    """
    Call the gameplay agent tool to get information about gameplay.
    """
    gameplay_agent_arn = get_agent_arn("gameplay_agent")
    response = json.loads(invoke_agent_with_boto3(gameplay_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']

@tool
def get_strategy_agent(payload) -> str:
    """
    Call the strategy agent tool to get information about corporate strategy.
    """
    strategy_agent_arn = get_agent_arn("strategy_agent")
    response = json.loads(invoke_agent_with_boto3(strategy_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']


@app.entrypoint
def game_analyst_agent(payload):
    global toolmetrics
    toolmetrics = []

    agent = Agent(
        tools=[get_lore_agent, get_strategy_agent, get_gameplay_agent],
        model=bedrock_model,
        system_prompt=GAME_ANALYST_PROMPT
    )

    try:
        user_input = payload.get("prompt")
        response = agent(user_input)
        fullmetrics = response.metrics.get_summary()
        output = {
            "response": response.message['content'][0]['text'],
            "metrics": fullmetrics,
            "toolMetrics": toolmetrics
        }
        return output
    except Exception as e:
        return f"error {e}"

if __name__ == "__main__":
    app.run()


### Launch the game analyst agent runtime

In [ ]:
_, game_analyst_agentcore_runtime = configure_runtime("game_analyst_agent_no_memories", game_analyst_iam_role, "game_analyst_agent.py")
game_analyst_launch_result = game_analyst_agentcore_runtime.launch(auto_update_on_conflict="true")

### Check the game analyst agent's endpoint status

In [ ]:
status = check_status(game_analyst_agentcore_runtime)
print(status)

### Test the game analyst agent

In [ ]:
invoke_response = game_analyst_agentcore_runtime.invoke({"prompt": "I want to introduce a portal system where users can talk to a witch and learn how to create portals. These portals can teleport them to any location that they have already been to"})
invoke_response

In [ ]:
invoke_response = game_analyst_agentcore_runtime.invoke({"prompt": "I have a new character idea that is an elf named Stark. She has magical healing powers that give her 50% increased healing rate and the ability to heal others. What do you think?"})
invoke_response

# Agent Memory

Maybe add a diagram here to show where memories are going

## Create Memories

### Create game analyst's short-term memory


In [ ]:
from botocore.exceptions import ClientError
import os
import boto3
import uuid
import logging
from bedrock_agentcore.memory import MemoryClient

def create_session_memory():
    # Configuration
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
    )

    logger = logging.getLogger("game-analyst-short-term-memory")
    REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
    memory_client = MemoryClient(region_name=REGION)

    # Create unique identifier for this resource
    unique_id = str(uuid.uuid4())[:8]
    memory_name = f"GameAnalystShortTermMemory_{unique_id}"

    try:
        # Create memory resource without strategies (short-term memory only)
        memory = memory_client.create_memory_and_wait(
            name=memory_name,
            strategies=[],  # No strategies for short-term memory
            description="Short-term memory for AgentCore Runtime agent",
            event_expiry_days=7, # Retention period for short-term memory
        )
        memory_id = memory['id']
        logger.info(f"✅ Created memory: {memory_id}")
        return memory_id
    except ClientError as e:
        logger.info(f"❌ ERROR: {e}")
        if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
            # If memory already exists, retrieve its ID
            memories = memory_client.list_memories()
            memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
            logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
    except Exception as e:
        # Show any errors during memory creation
        logger.error(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()
        # Cleanup on error - delete the memory if it was partially created
        if 'memory_id' in locals() and memory_id:
            try:
                memory_client.delete_memory_and_wait(memory_id=memory_id)
                logger.info(f"Cleaned up memory: {memory_id}")
            except Exception as cleanup_error:
                logger.error(f"Failed to clean up memory: {cleanup_error}")

In [ ]:
session_memory_id = create_session_memory()

### Create specialized agent long-term semantic memories

In [ ]:
from botocore.exceptions import ClientError
import os
import boto3
import uuid
import logging
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

def create_specialist_memories(role):
    # Configuration
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
    )

    logger = logging.getLogger("project-memory-agent")
    REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
    memory_client = MemoryClient(region_name=REGION)

    # Create unique identifier for this resource
    unique_id = str(uuid.uuid4())[:8]
    memory_name = f"Project_{role}_{unique_id}"
    namespace_prefix = f"project/{role}/"
    namespace = namespace_prefix+"{actorId}/semantic"

    try:
        memory = memory_client.create_memory_and_wait(
            name=memory_name,
            strategies=[{
                StrategyType.SEMANTIC.value: {
                    "name": "ProjectSemantic", 
                    "description": "Stores facts from conversations",
                    "namespaces": [namespace],
                }
            }],
            description="Project specific semantic memory for specialized agents",
            event_expiry_days=90, # Retention period for long-term memory
        )
        memory_id = memory['id']
        logger.info(f"✅ Created memory: {memory_id}")
        return memory_id
    except ClientError as e:
        logger.info(f"❌ ERROR: {e}")
        if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
            # If memory already exists, retrieve its ID
            memories = memory_client.list_memories()
            memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
            logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
    except Exception as e:
        # Show any errors during memory creation
        logger.error(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()
        # Cleanup on error - delete the memory if it was partially created
        if 'memory_id' in locals() and memory_id:
            try:
                memory_client.delete_memory_and_wait(memory_id=memory_id)
                logger.info(f"Cleaned up memory: {memory_id}")
            except Exception as cleanup_error:
                logger.error(f"Failed to clean up memory: {cleanup_error}")

In [ ]:
gameplay_project_memory_id = create_specialist_memories("gameplay")
lore_project_memory_id = create_specialist_memories("lore")
strategy_project_memory_id = create_specialist_memories("strategy")

In [ ]:
print(gameplay_project_memory_id)
print(lore_project_memory_id)
print(strategy_project_memory_id)

### Create game analyst user preferences long-term memories

In [ ]:
from botocore.exceptions import ClientError
import os
import boto3
import uuid
import logging
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType


def create_analyst_user_preferences():
    # Configuration
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
    )

    logger = logging.getLogger("game-analyst-user-prefs")
    REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
    memory_client = MemoryClient(region_name=REGION)

    # Create unique identifier for this resource
    unique_id = str(uuid.uuid4())[:8]
    memory_name = f"GameAnalystUserPrefs_{unique_id}"

    try:
        memory = memory_client.create_memory_and_wait(
            name=memory_name,
            strategies=[{
                StrategyType.USER_PREFERENCE.value: {
                    "name": "CustomerPreferences",
                    "description": "Captures customer preferences and behavior",
                    "namespaces": ["analyst/{actorId}/preferences"],
                }
            }],
            description="Project specific semantic memory for specialized agents",
            event_expiry_days=90, # Retention period for long-term memory
        )
        memory_id = memory['id']
        logger.info(f"✅ Created memory: {memory_id}")
        return memory_id
    except ClientError as e:
        logger.info(f"❌ ERROR: {e}")
        if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
            # If memory already exists, retrieve its ID
            memories = memory_client.list_memories()
            memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
            logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
    except Exception as e:
        # Show any errors during memory creation
        logger.error(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()
        # Cleanup on error - delete the memory if it was partially created
        if 'memory_id' in locals() and memory_id:
            try:
                memory_client.delete_memory_and_wait(memory_id=memory_id)
                logger.info(f"Cleaned up memory: {memory_id}")
            except Exception as cleanup_error:
                logger.error(f"Failed to clean up memory: {cleanup_error}")

In [ ]:
user_prefs_memory_id = create_analyst_user_preferences()

In [ ]:
print(user_prefs_memory_id)

### Create memory hook classes for Strands agents

look at memories.py

### For when you already have memories

In [ ]:
user_prefs_memory_id = "GameAnalystUserPrefs_37d1e66e-Ytm1ni3Db5"
gameplay_project_memory_id = "Project_gameplay_597edeef-XOt1OmF0Zd"
lore_project_memory_id = "Project_lore_78f09820-iidpdZ4wfj"
strategy_project_memory_id = "Project_strategy_3443cbe2-0vOWxMHl4Y"
session_memory_id = "GameAnalystShortTermMemory_6502d2be-wJVopQEE4H"

### Update Agent IAM Roles to use memories

In [ ]:
import boto3
import json

def update_agent_iam_roles_for_memories(session_memory_id, gameplay_project_memory_id,lore_project_memory_id, strategy_project_memory_id, user_prefs_memory_id, region="us-west-2"):
    """
    Update IAM roles for gameplay, lore, strategy, and game analyst agents to access memories
    """
    iam_client = boto3.client('iam')
    
    # Agent role names from the notebook
    agent_roles = [
       game_analyst_role_name,
        lore_agent_role_name, 
        strategy_role_name,
        gameplay_role_name
    ]
    
    # Memory policy for all three memory types
    memory_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:GetMemory",
                    "bedrock-agentcore:RetrieveMemoryRecords",
                    "bedrock-agentcore:GetMemoryRecord",
                    "bedrock-agentcore:CreateEvent",
                    "bedrock-agentcore:GetEvent",
                    "bedrock-agentcore:CreateMemory",
                    "bedrock-agentcore:DeleteMemoryRecord",
                    "bedrock-agentcore:DeleteMemory",
                    "bedrock-agentcore:UpdateMemory"
                ],
                "Resource": [
                    f"arn:aws:bedrock-agentcore:{region}:*:memory/{session_memory_id}",
                    f"arn:aws:bedrock-agentcore:{region}:*:memory/{gameplay_project_memory_id}",
                    f"arn:aws:bedrock-agentcore:{region}:*:memory/{lore_project_memory_id}",
                    f"arn:aws:bedrock-agentcore:{region}:*:memory/{strategy_project_memory_id}",
                    f"arn:aws:bedrock-agentcore:{region}:*:memory/{user_prefs_memory_id}"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:ListEvents",
                    "bedrock-agentcore:ListMemories",
                    "bedrock-agentcore:ListMemoryRecords",
                    "bedrock-agentcore:ListSessions"
                ],
                "Resource": [
                    "*"
                ]
		    }
        ]
    }
    
    # Update each agent role
    for role_name in agent_roles:
        try:
            iam_client.put_role_policy(
                RoleName=role_name,
                PolicyName="AgentCoreMemoryAccess",
                PolicyDocument=json.dumps(memory_policy)
            )
            print(f"✅ Updated {role_name} with memory access permissions")
        except Exception as e:
            print(f"❌ Failed to update {role_name}: {e}")

# Execute the update
update_agent_iam_roles_for_memories(session_memory_id, gameplay_project_memory_id,lore_project_memory_id, strategy_project_memory_id, user_prefs_memory_id)

## Update gameplay agent to use project memory

In [ ]:
# set the current working directory to be the gameplay-agent folder
import os
# os.chdir('strands_agents/gameplay-agent')
os.chdir('./strands-agents-with-memory/gameplay-agent-with-memory')
print(os.getcwd())

In [ ]:
gameplay_agent_code = f'''

import os
os.environ['OTEL_SDK_DISABLED'] = 'true'
os.environ['OTEL_PYTHON_DISABLED_INSTRUMENTATIONS'] = 'botocore,boto3'

import argparse
import json
import logging
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider
from memories import LTMemoryHookProvider
from bedrock_agentcore.memory import MemoryClient

# Uninstrument botocore to prevent OpenTelemetry crashes
try:
    from opentelemetry.instrumentation.botocore import BotocoreInstrumentor
    BotocoreInstrumentor().uninstrument()
except:
    pass

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger('bedrock_agentcore').setLevel(logging.ERROR)
logging.getLogger('strands').setLevel(logging.ERROR)


###########################
# MCP Clients             #
###########################

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

# System prompt for the Gameplay agent
GAMEPLAY_AGENT_PROMPT = """
# New World Gameplay Expert Assistant

## Role Definition
You are an authoritative New World Gameplay Expert specializing in Amazon Game Studios' New World MMORPG. Your purpose is to provide accurate, helpful information about all game mechanics and systems.

## Areas of Expertise
<expertise>
- **Character Development**: Attributes (Strength, Dexterity, Intelligence, Focus), weapon scaling, builds
- **Crafting Systems**: Armoring, Weaponsmithing, Engineering, Arcana, Cooking, Furnishing
- **Gathering & Refining**: Mining, Logging, Harvesting, Fishing, Skinning, Smelting, Woodworking, Leatherworking
- **Faction Systems**: Covenant, Marauders, Syndicate mechanics, territory control, faction missions
- **PvE Content**: Expeditions, elite zones, corrupted breaches, outpost rush
- **Game Mechanics**: Combat systems, perks, gems, optimization strategies, meta builds
</expertise>

## Information Retrieval Protocol
Follow this strict sequence for every query:

1. **FIRST**: Use memory retrieval tools to search for the requested information
   - Query format: [Search terms directly related to user question]
   - Example: If asked about "best strength weapons", search for "strength weapons scaling New World"

2. **Information Processing**:
   - If memory tools return relevant information and information is sufficient → respond using retrieved information
   - If memory tools return no/insufficient information → ONLY THEN use knowledge base tools
   - If using knowledge base, search with specific terms from the user's question

3. **Response Structure**:
   - Clearly indicate source: "Based on stored information..." or "According to the knowledge base..."
   - Provide practical, actionable gameplay advice
   - Include specific game mechanics details when available
   - Format information in an easily digestible way (bullet points for lists, bold for key terms)

## Critical Rules
- NEVER use knowledge base tools without first attempting memory retrieval
- ONLY store memories of factual information retrieved from knowledge base tools
- Do not speculate beyond the information available in your tools
- If information conflicts, prioritize the most recent data
- If you cannot find reliable information, acknowledge limitations rather than guessing

Respond to the user's New World gameplay question with accurate, helpful information following the protocol above. Provide your answer in a clear, concise format without any preamble.
"""

app = BedrockAgentCoreApp()

@app.entrypoint
def gameplay_agent(payload, context):
    """
    Creates and runs a gameplay agent connected to the New World Knowledge Base with memory
    """
    try:
        client = MemoryClient(region_name="us-west-2")
        # Knowledge Base MCP Client
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "new-world-design-docs"
                    }
                )

            )
        )
    
        # Memory setup
        project_memory_id = "Project_gameplay_597edeef-XOt1OmF0Zd"
        print(f"Project memory ID: {gameplay_project_memory_id}")

        # Get user id
        # user_id = payload.get("user_id", "default-user")

        # Get project id for user project memory
        project_id = payload.get("project_id", "new-world-aternum")

        # Get session id for short-term memory
        session_id = context.session_id  # Get session_id from context

        # Namepspace for project memory for gameplay agent 
        gameplay_namespace = f"project/gameplay/{{project_id}}/semantic"
        
        # Memory tool provider to retrieve memories
        project_memory_provider = AgentCoreMemoryToolProvider(
            memory_id=project_memory_id,
            actor_id=project_id,
            session_id=session_id,
            namespace=gameplay_namespace
        )

        # Hook provider to link actions to a event in the agent lifecycle
        # This is how we define how the agent will store events in its memory
        project_memory_hooks = LTMemoryHookProvider(project_memory_id, client)
        
        with kb_client:
            # Get tools from KB client and memory
            kb_tools = kb_client.list_tools_sync()
            all_tools = kb_tools + project_memory_provider.tools

            # Create the gameplay agent with memory
            agent = Agent(
                tools=all_tools,
                model=bedrock_model,
                hooks=[project_memory_hooks],
                system_prompt=GAMEPLAY_AGENT_PROMPT,
                state={{"actor_id": project_id, "session_id": session_id}}
            )
            
            user_input = payload.get("prompt")
            print("User input:", user_input)
            response = agent(user_input)
            
            return {{
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }}
            
    except Exception as e:
        print(f"Error: {{e}}")
        return f"error {{e}}"

if __name__ == "__main__":
    app.run()
'''

with open('gameplay_agent.py', 'w') as f:
    f.write(gameplay_agent_code)

### Re-launch the gameplay agent runtime

In [ ]:
_, gameplay_agentcore_runtime = configure_runtime("gameplay_agent", gameplay_iam_role, "gameplay_agent.py")
gameplay_launch_result = gameplay_agentcore_runtime.launch(auto_update_on_conflict="true")
gameplay_agent_id = gameplay_launch_result.agent_id
gameplay_agent_arn = gameplay_launch_result.agent_arn

print(gameplay_agent_arn)

### Save the gameplay agent ARN to AWS SSM ParameterStore

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/gameplay_agent_arn',
    Value=gameplay_agent_arn,
    Type='String',
    Overwrite=True
)

### Check the updated gameplay agent's endpoint status

In [ ]:
status = check_status(gameplay_agentcore_runtime)
print(status)

### Test the updated gameplay agent

In [ ]:
invoke_response = gameplay_agentcore_runtime.invoke({"prompt": "Identify the best weapon combinations for a Dexterity build in New World speceifically geared towards PvP combat.", "user_id": "test-user-126","project_id": "new-world-aternum", "session_id": "test-session-002"})
invoke_response

In [ ]:
invoke_response = gameplay_agentcore_runtime.invoke({"prompt": "What are the best weapon combinations for a Dexterity build in New World speceifically geared towards PvP combat.", "user_id": "test-user-123","project_id": "new-world-aternum", "session_id": "test-session-001"})
invoke_response

## Update the lore agent to use project memories

In [ ]:
os.chdir('./strands-agents-with-memory/lore-agent-with-memory')
print(os.getcwd())

In [ ]:
lore_agent_code = f'''
import os
os.environ['OTEL_SDK_DISABLED'] = 'true'
os.environ['OTEL_PYTHON_DISABLED_INSTRUMENTATIONS'] = 'botocore,boto3'

import argparse
import json
import logging
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider
from memories import LTMemoryHookProvider
from bedrock_agentcore.memory import MemoryClient

try:
    from opentelemetry.instrumentation.botocore import BotocoreInstrumentor
    BotocoreInstrumentor().uninstrument()
except:
    pass

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger('bedrock_agentcore').setLevel(logging.ERROR)
logging.getLogger('strands').setLevel(logging.ERROR)

###########################
# MCP Clients             #
###########################

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

# System prompt for the Knowledge Base agent
LORE_AGENT_PROMPT = """
# New World Game Lore Expert

<role>
You are an authoritative Game Analyst specializing in the lore and world-building of "New World," the MMORPG developed by Amazon Game Studios. Your purpose is to provide accurate, comprehensive information about the game's narrative elements, history, and world.
</role>

- Complete understanding of New World's core storyline, main objectives, and narrative arcs
- Detailed knowledge of Aeternum's history, mythology, and underlying lore
- Comprehensive information about locations, geographical features, settlements, and territories
- Familiarity with key characters, factions, and demographic groups within the game world
</expertise>

<information_retrieval_protocol>
When responding to any user query about New World lore, strictly follow this sequence:

1. **MEMORY RETRIEVAL (MANDATORY FIRST STEP)**
   - Search your memory using specific terms directly related to the user's question
   - Example search format: [Aeternum origin] [Angry Earth faction] [Great Cleave history]

2. **INFORMATION PROCESSING**
   - If memory tools return relevant information and information is sufficient → respond using retrieved information
   - If memory retrieval yields insufficient or no information → ONLY THEN access your knowledge base
   - When using knowledge base, search with precise terms from the user's question

3. **SOURCE ATTRIBUTION**
   - Clearly indicate your information source in your response:
     * "Based on stored information..." (when using memory)
     * "According to the knowledge base..." (when using knowledge base)
</information_retrieval_protocol>

<response_guidelines>
- Present information in a structured, easily digestible format
- Use **bold formatting** for key terms, locations, and character names
- Organize lists and multiple points using bullet points
- Include specific in-game details when available
- Provide context that helps players understand the narrative significance
</response_guidelines>

<critical_rules>
- NEVER access knowledge base without first attempting memory retrieval
- Only store factual information retrieved from knowledge base tools as memories
- Do not speculate or invent lore beyond what's available in your tools
- When information conflicts, prioritize the most recent or most authoritative source
- If you cannot find reliable information on a topic, acknowledge these limitations rather than guessing
</critical_rules>

When a user asks about New World lore, respond with accurate, helpful information following this protocol. Your answer should be clear, concise, and directly address the user's question without any unnecessary preamble.
"""
app = BedrockAgentCoreApp()
# Create and run the Knowledge Base agent
@app.entrypoint
def lore_agent(payload, context):
    """
    Creates and runs an agent connected to the Knowledge Base
    """
    # Knowledge Base MCP Client
    try:
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "new-world-design-docs"
                    }
                )

            )
        )

        # Initialize the memory client
        client = MemoryClient(region_name="us-west-2")

        # Memory setup
        project_memory_id = "Project_lore_78f09820-iidpdZ4wfj"
        print(f"Project memory ID: {lore_project_memory_id}")
        user_id = payload.get("user_id", "default-user")
        project_id = payload.get("project_id", "new-world-aternum")
        session_id = context.session_id
        lore_namespace = f"project/{{project_id}}/semantic"
        
        # Memory tool provider to retrieve memories
        project_memory_provider = AgentCoreMemoryToolProvider(
            memory_id=project_memory_id,
            actor_id=project_id,
            session_id=session_id,
            namespace=lore_namespace
        )

        # Hook provider to save memories
        project_memory_hooks = LTMemoryHookProvider(project_memory_id, client)

        with kb_client:
            # Get tools from the KB client
            kb_tools = kb_client.list_tools_sync()
            
            all_tools = kb_tools + project_memory_provider.tools

            # Create the gameplay agent with memory
            agent = Agent(
                tools=all_tools,
                model=bedrock_model,
                hooks=[project_memory_hooks],
                system_prompt=LORE_AGENT_PROMPT,
                state={{"actor_id": project_id, "session_id": session_id}}
            )
            
            user_input = payload.get("prompt")
            print("User input:", user_input)
            response = agent(user_input)
            return {{
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }}
        
                    
    except Exception as e:
        print(f"Error accessing Knowledge Base: {{e}}")
        return f"error {{e}}"

if __name__ == "__main__":
    app.run()
'''

with open('lore_agent.py', 'w') as f:
    f.write(lore_agent_code)

### Re-launch the lore agent runtime

In [ ]:
_, lore_agentcore_runtime = configure_runtime("lore_agent", lore_agent_iam_role, "lore_agent.py")
lore_launch_result = lore_agentcore_runtime.launch(auto_update_on_conflict="true")
lore_agent_id = lore_launch_result.agent_id
lore_agent_arn = lore_launch_result.agent_arn

print(lore_agent_arn)

### Save the agent ARN to AWS SSM ParameterStore

In [ ]:
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/lore_agent_arn',
    Value=lore_agent_arn,
    Type='String',
    Overwrite=True
)

### Check the updated lore agent's endpoint status

In [ ]:
status = check_status(lore_agentcore_runtime)
print(status)

### Test the updated lore agent

In [ ]:
invoke_response = lore_agentcore_runtime.invoke({"prompt": "How did the Angry Earth come to be, and what is their relationship with the island's natural Azoth energy?", "user_id": "test-user-125","project_id": "new-world-aternum", "session_id": "test-session-005"})
invoke_response

In [ ]:
invoke_response = lore_agentcore_runtime.invoke({"prompt": "How did the Angry Earth come to be?", "user_id": "test-user-128","project_id": "new-world-aternum", "session_id": "test-session-009"})
invoke_response

## Update the strategy agent to use project memories

In [ ]:
os.chdir('./strands-agents-with-memory/strategy-agent-with-memory')
print(os.getcwd())

In [ ]:
strategy_agent_code = f'''
import os
os.environ['OTEL_SDK_DISABLED'] = 'true'
os.environ['OTEL_PYTHON_DISABLED_INSTRUMENTATIONS'] = 'botocore,boto3'

import argparse
import json
import logging
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider
from memories import LTMemoryHookProvider
from bedrock_agentcore.memory import MemoryClient

try:
    from opentelemetry.instrumentation.botocore import BotocoreInstrumentor
    BotocoreInstrumentor().uninstrument()
except:
    pass

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger('bedrock_agentcore').setLevel(logging.ERROR)
logging.getLogger('strands').setLevel(logging.ERROR)

###########################
# MCP Clients             #
###########################

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)

# System prompt for the Knowledge Base agent
STRATEGY_AGENT_PROMPT = """
You are a business strategy expert at Amazon Game Studios. Your expertise includes:
- Game industry market analysis and competitive positioning
- Player acquisition, retention, and monetization strategies
- Product portfolio management and roadmap planning
- Performance metrics analysis and strategic recommendations
- Resource allocation and investment prioritization
- Company priorities and roadmaps


You have access to a knowledge base containing corporate strategy documents of Amazon Game Studios. ALWAYS use the available knowledge base tools to retrieve information before answering questions. If the knowledge base returns relevant information, use that information in your response.

When using knowledge base tools, make sure to:
1. Use the exact query or related terms to search the knowledge base
2. Include the retrieved information in your response
3. Only use information from the knowledge base
"""
app = BedrockAgentCoreApp()
# Create and run the Knowledge Base agent
@app.entrypoint
def strategy_agent(payload, context):
    """
    Creates and runs an agent connected to the Knowledge Base
    """
    try:
        # Knowledge Base MCP Client
        kb_client = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx", 
                    args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
                    env={
                        "BEDROCK_KB_RERANKING_ENABLED": "true",
                        "KB_INCLUSION_TAG_KEY": "strategy-docs"
                    }
                )

            )
        )

        # Initialize the memory client
        client = MemoryClient(region_name="us-west-2")

        # Memory setup
        project_memory_id = "Project_strategy_3443cbe2-0vOWxMHl4Y"
        print(f"Project memory ID: {strategy_project_memory_id}")
        user_id = payload.get("user_id", "default-user")
        project_id = payload.get("project_id", "new-world-aternum")
        session_id = context.session_id
        strategy_namespace = f"project/{{project_id}}/semantic"
        
        # Memory tool provider to retrieve memories
        project_memory_provider = AgentCoreMemoryToolProvider(
            memory_id=project_memory_id,
            actor_id=project_id,
            session_id=session_id,
            namespace=strategy_namespace
        )

        # Hook provider to save memories
        project_memory_hooks = LTMemoryHookProvider(project_memory_id, client)

        with kb_client:
            # Get tools from the KB client
            kb_tools = kb_client.list_tools_sync()
            
            all_tools = kb_tools + project_memory_provider.tools

            # Create the gameplay agent with memory
            agent = Agent(
                tools=all_tools,
                model=bedrock_model,
                hooks=[project_memory_hooks],
                system_prompt=STRATEGY_AGENT_PROMPT,
                state={{"actor_id": project_id, "session_id": session_id}}
            )
            
            user_input = payload.get("prompt")
            print("User input:", user_input)
            response = agent(user_input)
            return {{
                "response": response.message['content'][0]['text'],
                "metrics": response.metrics.get_summary()
            }}
        
                    
    except Exception as e:
        print(f"Error accessing Knowledge Base: {{e}}")
        return f"error {{e}}"

if __name__ == "__main__":
    app.run()

'''

with open('strategy_agent.py', 'w') as f:
    f.write(strategy_agent_code)

### Re-Launch the srategy agent runtime

In [ ]:
_, strategy_agentcore_runtime = configure_runtime("strategy_agent", strategy_iam_role, "strategy_agent.py")
strategy_launch_result = strategy_agentcore_runtime.launch(auto_update_on_conflict="true")
strategy_agent_id = strategy_launch_result.agent_id
strategy_agent_arn = strategy_launch_result.agent_arn

print(strategy_agent_arn)

### Save the agent ARN to AWS SSM ParameterStore

In [ ]:
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/strategy_agent_arn',
    Value=strategy_agent_arn,
    Type='String',
    Overwrite=True
)

### Check the updated strategy agent's endpoint status

In [ ]:
status = check_status(strategy_agentcore_runtime)
print(status)

### Test the updated strategy agent

In [ ]:
invoke_response = strategy_agentcore_runtime.invoke({"prompt": "What is the main goal for new characters for 2026?", "user_id": "test-user-128","project_id": "new-world-aternum", "session_id": "test-session-009"})
invoke_response

## Update game analyst agent to include all user preferences and session memories

In [ ]:
os.chdir('./strands-agents-with-memory/game-analyst-agent-with-memory')
print(os.getcwd())

In [ ]:
game_analyst_agent_code = f'''
import os
os.environ['OTEL_SDK_DISABLED'] = 'true'
os.environ['OTEL_PYTHON_DISABLED_INSTRUMENTATIONS'] = 'botocore,boto3'

import json
import logging
import boto3
from typing import Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from invoke_agent_utils import invoke_agent_with_boto3
from memories import ShortTermMemoryHook as STM, LTMemoryHookProvider as LTM
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

try:
    from opentelemetry.instrumentation.botocore import BotocoreInstrumentor
    BotocoreInstrumentor().uninstrument()
except:
    pass

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger('bedrock_agentcore').setLevel(logging.ERROR)
logging.getLogger('strands').setLevel(logging.ERROR)

# Initialize the agent core app
app = BedrockAgentCoreApp()

toolmetrics = []

# Configure Bedrock model
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    temperature=0.3,
)


# System prompt for the Game Analyst agent
GAME_ANALYST_PROMPT = """
PLEASE PRINT AT THE TOP IF YOU DONT USE THE TOOLS and EXPAND ON WHY YOU DIDNT. IF YOU DO USE THE TOOLS ANNOTATE WHERE.

# Game Analyst Agent

## Role and Expertise
You are a professional Game Analyst Agent specializing in analyzing game design documents for New World Aeternum (also known as New World). Your expertise includes understanding the game's story, lore, gameplay mechanics, and overall design philosophy.

## Primary Objectives
- Review game design documentation thoroughly
- Understand the developer's vision and intentions
- Provide expert feedback with insightful analysis
- Offer constructive recommendations that maintain consistency with the game's established elements
- Remember user preferences from previous interactions in the same session

## Available Tools
You have access to specialized tools to enhance your analysis:

<tool_lore>
**Lore Agent Tool**
Use this tool for queries about the game world, story, and lore:
- Narrative consistency and world-building evaluation
- Character development analysis
- Thematic elements and story structure assessment
- Fictional histories and setting coherence verification
- Identification of opportunities for deeper player immersion
</tool_lore>

<tool_gameplay>
**Gameplay Agent Tool**
Use this tool for queries about game mechanics and gameplay systems:
- Game systems, mechanics, and balance analysis
- Core gameplay loops and progression systems evaluation
- Player motivation and engagement assessment
- Difficulty curves, skill ceilings, and accessibility considerations
- Identification of potential balance issues and optimization opportunities
</tool_gameplay>

<tool_strategy>
**Corporate Strategy Agent Tool**
For queries around corporate strategy, performance metrics, roadmaps, and strategic priorities use strategy_agent tool:
    - Specializes in corporate strategy, performance metrics, and strategic priorities
    - Analyzes corporate strategy, performance metrics, and strategic priorities
    - Evaluates corporate strategy, performance metrics, and strategic priorities
    - Identifies opportunities for improvement in corporate strategy, performance metrics, and strategic priorities
</tool_strategy>

## Memory Functionality
You can recall previous interactions across sessions within the previous 7 days. Always refer to your memory regarding user preferences for formatting your responses and their specific interests or concerns about their game design. You may refer to previously saved memories on session data to answer questions.

## Response Guidelines
1. Analyze the submitted game design document thoroughly
2. Maintain consistency with New World's established story, lore, and gameplay
3. Provide specific, actionable feedback rather than general statements
4. Format your response according to previously established user preferences

When responding to queries, provide your analysis directly without preambles or additional explanations beyond what was requested.
"""

def get_agent_arn(agent_name: str) -> str:
    """
    Retrieve agent ARN from Parameter Store
    """
    ssm = boto3.client('ssm')
    response = ssm.get_parameter(Name=f'/agents/{{agent_name}}_arn')
    return response['Parameter']['Value']


@tool
def get_lore_agent(payload) -> str:
    """
    Call the lore agent tool to get information about the game lore.
    """
    lore_agent_arn = get_agent_arn("lore_agent")
    response = json.loads(invoke_agent_with_boto3(lore_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']

@tool
def get_gameplay_agent(payload) -> str:
    """
    Call the gameplay agent tool to get information about gameplay.
    """
    gameplay_agent_arn = get_agent_arn("gameplay_agent")
    response = json.loads(invoke_agent_with_boto3(gameplay_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']

@tool
def get_strategy_agent(payload) -> str:
    """
    Call the strategy agent tool to get information about corporate strategy.
    """
    strategy_agent_arn = get_agent_arn("strategy_agent")
    response = json.loads(invoke_agent_with_boto3(strategy_agent_arn, user_query=payload))
    toolmetrics.append(response['metrics'])
    return response['response']

@app.entrypoint
def game_analyst_agent(payload, context):
    global toolmetrics
    toolmetrics = []
    user_input = payload.get("prompt")
    user_id = payload.get("user_id", "default-user")
    project_id = payload.get("project_id", "new-world-aternum")
    session_id = context.session_id
    
    client = MemoryClient(region_name="us-west-2")
    
    # Memory setup
    user_prefs_memory_id = "GameAnalystUserPrefs_37d1e66e-Ytm1ni3Db5"
    user_prefs_namespace = f"analyst/{{user_id}}/preferences"

    # Memory tool provider
    user_prefs_memory_provider = AgentCoreMemoryToolProvider(
        memory_id=user_prefs_memory_id,
        actor_id=user_id,
        session_id=session_id,
        namespace=user_prefs_namespace
    )
    session_memory_id = "GameAnalystShortTermMemory_6502d2be-wJVopQEE4H"

    user_prefs_memory_hooks = LTM(user_prefs_memory_id, client)
    session_memory_hooks = STM(client, session_memory_id)

    agent = Agent(
        tools=[get_gameplay_agent, get_lore_agent, get_strategy_agent] + user_prefs_memory_provider.tools,
        model=bedrock_model,
        hooks=[user_prefs_memory_hooks, session_memory_hooks],
        system_prompt=GAME_ANALYST_PROMPT,
        state={{"actor_id": user_id, "session_id": session_id}}
    )
    
    try:
        response = agent(user_input)
        
        return {{
            "response": response.message['content'][0]['text'],
            "metrics": response.metrics.get_summary(),
            "toolMetrics": toolmetrics
        }}
    except Exception as e:
        return {{"error": str(e)}}
    

if __name__ == "__main__":
    app.run()
'''

with open('game_analyst_agent.py', 'w') as f:
    f.write(game_analyst_agent_code)

### Re-launch the updated game analyst agent runtime

In [ ]:
_, game_analyst_agentcore_runtime = configure_runtime("game_analyst_agent", game_analyst_iam_role, "game_analyst_agent.py")
game_analyst_launch_result = game_analyst_agentcore_runtime.launch(auto_update_on_conflict="true")

### Check the updated game analyst agent's endpoint status

In [ ]:
status = check_status(game_analyst_agentcore_runtime)
print(status)

### Test the updated game analyst agent

In [ ]:
import uuid
test_session_id = "test-session-12345676809453903427"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "I want to introduce a portal system where users can talk to a witch and learn how to create portals. These portals can teleport them to any location that they have already been to",
        "project_id": "new-world-aternum",
        "user_id": "test_user_129"
    }, 
    session_id=test_session_id
)

invoke_response

In [ ]:
import uuid
test_session_id = "test-session-12345676809453903827"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "Hello",
        "project_id": "new-world-aternum",
        "user_id": "test_user_120"
    }, 
    session_id=test_session_id
)
invoke_response

In [ ]:
import uuid
test_session_id = "test-session-12345676809453903827"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "I want to introduce a portal system where users can talk to a witch and learn how to create portals. These portals can teleport them to a robot world where they can master a new high tech weapon mastery tree.",
        "project_id": "new-world-aternum",
        "user_id": "test_user_120"
    }, 
    session_id=test_session_id
)

invoke_response

In [ ]:
import uuid
test_session_id = "test-session-12345676809453903827"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "What if I made the world the portals led to instead a land of elves?",
        "project_id": "new-world-aternum",
        "user_id": "test_user_120"
    }, 
    session_id=test_session_id
)

invoke_response

In [ ]:
import uuid
test_session_id = "test-session-12345676809453900499"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "Proposing a new item called the holy grail where if it is found the player is resistent to any damage for 30 seconds when the item is activated. It can only be activated once per raid.",
        "project_id": "new-world-aternum",
        "user_id": "test_user_189"
    }, 
    session_id=test_session_id
)

invoke_response

In [ ]:
import uuid
test_session_id = "test-session-12345676809453903410"

# Send our first message
invoke_response = game_analyst_agentcore_runtime.invoke(
    {
        "prompt": "A portal system that is summoned by an ancient that you meet up in a mountain. They think you stole their precious stone so they send you through the portal to a mystical realm that you must find a way to escape.",
        "project_id": "new-world-aternum",
        "user_id": "test_user_139"
    }, 
    session_id=test_session_id
)

invoke_response